In [1]:
import _threshold_utils as threshold_utils
import _speed_utils as speed_utils
import altair as alt

import pandas as pd
import geopandas as gpd
from segment_speed_utils.project_vars import analysis_date
from shared_utils import calitp_color_palette as cp

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/jovyan/data-analyses/rt_segment_speeds/_threshold_utils.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/lat

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/"

In [4]:
analysis_date

'2023-03-15'

### Functions

In [6]:
# Flag
routes_many_stops_df, routes_many_stops_list = speed_utils.find_shapes_with_many_stops(analysis_date)

14517 duplicate rows will be dropped
0    4512
Name: stops_renumbered_correctly, dtype: int64


In [9]:
routes_many_stops_df.shape

(3806, 6)

In [10]:
routes_many_stops_df.head()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,trip_id,total_stops,stop_percentiles
0,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,t604-b2791-sl5,640,99th 203.0+ stops
1,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,t4D1-b27B5-sl5,616,99th 203.0+ stops
2,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,t59B-b2793-sl5,614,99th 203.0+ stops
3,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,t668-b278A-sl5,611,99th 203.0+ stops
4,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,t64A-b275D-sl5,611,99th 203.0+ stops


In [8]:
m1 =  speed_utils.merge_all_speeds(analysis_date)

14517 duplicate rows will be dropped
0    4512
Name: stops_renumbered_correctly, dtype: int64


In [ ]:
def open_speeds_stops(date: str)-> pd.DataFrame:
    """
    Open up speeds_stops parquet.
    
    Args:
        date: analysis date
    """
    speed_stops_subset = ['gtfs_dataset_key', '_gtfs_dataset_name', 'shape_array_key',
       'stop_sequence', 'trip_id','speed_mph']
    
    df = pd.read_parquet(f"{GCS_PATH}speeds_stop_segments_{date}")
    
    df = df[speed_stops_subset]
    
    return df

In [ ]:
def open_avg_speeds(date:str)-> pd.DataFrame:
    """
    Open up avg_speeds_stop_segments parquet.
    
    Args:
        date: analysis date
    """
    df = gpd.read_parquet(f"{GCS_PATH}avg_speeds_stop_segments_{date}.parquet")
    cols_to_drop = ['geometry','geometry_arrowized','district','district_name']
    df = df.drop(columns = cols_to_drop)
    
    return df

In [ ]:
def merge_all_speeds(date:str) -> pd.DataFrame:
    """
    Merge avg_speeds_stop_segments and
    speed_stops parquets, drop duplicates, and
    renumber stop sequences that are out of order. 
    
    Args:
        date: analysis date
    """
    avg_speeds = open_avg_speeds(date)
    speed_stops = open_speeds_stops(date)
    
    merge_cols = ['gtfs_dataset_key','shape_array_key', 'stop_sequence']
    m1 = pd.merge(avg_speeds, speed_stops, on = merge_cols, how = 'inner')
    print(f"{len(m1) - len(m1.drop_duplicates())} duplicate rows will be dropped") 
    
    m1 = m1.drop_duplicates().reset_index(drop = True)
    
    # Renumber stop sequences since some of them are out of order. 
    # Ex: instead of stop 1,2,3 it could be stop 151,169, 300
    # Sort df first.
    sort_cols = ['shape_array_key', 'gtfs_dataset_key', 'trip_id', 'stop_sequence']
    m1 = m1.sort_values(sort_cols).reset_index(drop = True)
    
    # Count stops by operator (gtfs key), route (shape array), and trip
    group_cols = ['gtfs_dataset_key','shape_array_key', 'trip_id']
    m1['sorted_stop_seq'] = m1.groupby(group_cols).cumcount().astype(int)+1
    
    # Check that stops were renumbered correctly. Should only be 0.
    # Total original stops should equal newly renumbered ones. 
    check1 = m1.groupby(['shape_array_key']).agg({'sorted_stop_seq':'count', 'stop_sequence':'count'}).reset_index(drop = True)
    check1["stops_renumbered_correctly"] = check1.sorted_stop_seq-check1.stop_sequence
    print(check1.stops_renumbered_correctly.value_counts())
    
    return m1

In [ ]:
m1 = merge_all_speeds(analysis_date)

### See which routes have a lot of stops
* every shape has maybe 20, 30, 50 stops

In [ ]:
m1.stop_sequence.describe()

In [ ]:
m1.sorted_stop_seq.describe()

In [ ]:
# Check out routes with a ton of unique stops in the sequence.
m1.groupby(['gtfs_dataset_key','shape_array_key','trip_id']).agg({'sorted_stop_seq':'nunique'}).sort_values(['sorted_stop_seq'], ascending = False).head(15)

#### Look at one trip

In [ ]:
#foothill_og = speed_stops2[speed_stops2.trip_id == "t604-b2791-sl5"]

In [ ]:
#len(foothill_og)

In [ ]:
#foothill_og.stop_sequence.nunique()

In [ ]:
#foothill_og.stop_sequence.describe()

In [ ]:
#foothill_og.sort_values('stop_sequence').head()

In [ ]:
#foothill_renumbered_stop_seq = m2[m2.trip_id == "t604-b2791-sl5"] 

In [ ]:
#foothill_renumbered_stop_seq['Test Stop Sequence'].describe()

In [ ]:
#foothill_renumbered_stop_seq.sort_values('stop_sequence').head()

In [ ]:
# len(troubleshoot)

In [ ]:
# Number of test stops should match stop sequence...
# troubleshoot['sequences_are_equal'] = troubleshoot['Test Stop Sequence'] - troubleshoot['stop_sequence']

In [ ]:
# troubleshoot['sequences_are_equal'].value_counts()

In [ ]:
# Look at this trip id in the original df
#og_trip = speed_stops2[speed_stops2.trip_id == "t640-b15FF1-sl5"]

In [ ]:
# Look at this trip id in the manipulated df
#new_trip = m2[m2.trip_id == "t640-b15FF1-sl5"]

In [ ]:
#og_trip.shape, og_trip.stop_sequence.nunique()

In [ ]:
#new_trip.shape, new_trip.stop_sequence.nunique()

#### Function

In [ ]:
"""
p25 = troubleshoot.total_stops.quantile(0.25).astype(float)
p50 =  troubleshoot.total_stops.quantile(0.50).astype(float)
p75 =  troubleshoot.total_stops.quantile(0.75).astype(float)
p95 =  troubleshoot.total_stops.quantile(0.95).astype(float)
p99 =  troubleshoot.total_stops.quantile(0.99).astype(float)
"""

In [ ]:
def find_shapes_with_many_stops(date:str) -> pd.DataFrame:
    """
    Find the routes with total stops 
    in the 95th+ percentile.
    
    Args:
        date: analysis_date
        
    Returns:
        DF with the total stops by shape_array_key,
        gtfs_dataset_key, trip_id and total stops to check
        out routes.
        
        List of unique shape_array_keys.
    """
    df = merge_all_speeds(date)
    
    # Find the total number of stops by operator, shape_array_key (route), and trip_id
    agg = (df.groupby(['gtfs_dataset_key','_gtfs_dataset_name','shape_array_key','trip_id'])
                .agg({'stop_sequence':'nunique'})
                .sort_values(['stop_sequence'], ascending = False)
                .reset_index()
                .rename(columns = {'stop_sequence':'total_stops'})
               )
    
    # Categorize 
    p95 =  agg.total_stops.quantile(0.95).astype(float)
    p99 =  agg.total_stops.quantile(0.99).astype(float)
    
    def stop_categories(row):
        if ((row.total_stops >= p95) and (row.total_stops <= p99)):
               return f"95th {p95}-{p99} stops"
        elif row.total_stops > p99:
               return f"99th {p99}+ stops"
        else:
            return "other"
        
    agg["stop_percentiles"] = agg.apply(lambda x: stop_categories(x), axis=1)
    
    # Keep only the routes in the 95th percentile of stops
    agg2 = agg[agg.stop_percentiles.isin([f"95th {p95}-{p99} stops", f"99th {p99}+ stops"])].reset_index(drop = True)
    
    return agg2, agg2.shape_array_key.unique().tolist()

In [ ]:
routes_with_many_stops.shape

In [ ]:
routes_with_many_stops.shape_array_key.nunique()

In [ ]:
    # Summarize this: find median stops by shape array ID
    """
    agg3 = (agg2
                  .groupby(['_gtfs_dataset_name','shape_array_key', 'stop_percentiles'])
                  .agg({'trip_id':'nunique','total_stops':'median'})
                  .reset_index()
                  .rename(columns = {'total_stops':'total_stops_median'})
                 )
    display(agg3)
    """

### Charts  
Test with a few routes first
* Create new col that rounds up speed for plotting purposes only.

#### Manipulate DF for charts

In [ ]:
test1 = m1.melt(id_vars=[ '_gtfs_dataset_name','shape_array_key','trip_id', 'sorted_stop_seq','gtfs_dataset_key','loop_or_inlining',
                       'n_trips'], value_vars=[ 'avg_speed_mph','speed_mph','p20_speed_mph', 'p80_speed_mph'])

In [ ]:
test1.shape

In [ ]:
# test1[test1.shape_array_key == "29d2bbdbeaec1d6888800f85bebf6e33"]

In [ ]:
# Only need average speed/p20 speed/p80 to show up once for each stop sequence-operator-shape array
test2 = test1.drop_duplicates(subset = [ '_gtfs_dataset_name','shape_array_key','sorted_stop_seq','gtfs_dataset_key','variable','value']).reset_index(drop = True)

In [ ]:
test2.shape

In [ ]:
other = ['cf688717cf0cd8dac0e6d1f12f9c7333',
       '6f39f818c9a0c5496cd1c8bd1aa11e67',
       '3de4482ec32ba0f2edb451d3528b5a5e']

In [ ]:
subset = test2[test2.shape_array_key.isin(["29d2bbdbeaec1d6888800f85bebf6e33",'754c5b012195800c38dc58e72e4f482e',
       'e3c5ed2c6fa6cd5c5cd57d46aeb3cd8e', ])]

In [ ]:
subset = threshold_utils.pre_clean(subset)

In [ ]:
subset.columns

In [ ]:
subset['Route'] = subset['Gtfs Dataset Name'] +' '  + subset['Shape Array Key']

In [ ]:
subset = subset.rename(columns = {'Value':'Speed'})

In [ ]:
subset['Speed_Int'] = subset.Speed.fillna(0).astype(int)

In [ ]:
# subset['Rounded Speed'].unique()

In [ ]:
def speed(row):
    # If partner is none, return Unknown.
    if row.Speed_Int == 0:
        return 0
    elif 0 < row.Speed_Int < 6:
        return 5
    elif 5 < row.Speed_Int < 11:
        return 10
    elif 10 < row.Speed_Int < 16:
        return 15
    elif 15 < row.Speed_Int < 21:
        return 20
    elif 20 < row.Speed_Int < 26:
        return 25
    elif 25 < row.Speed_Int < 31:
        return 30
    else:
        return 35

In [ ]:
# Apply the function
subset["Rounded Speed"] = subset.apply(speed, axis=1)

In [ ]:
subset.groupby(['Shape Array Key', 'Trip Id']).agg({'Sorted Stop Seq':'count'})

In [ ]:
# subset[['Rounded Speed', 'Speed', 'Speed_Int']]

In [ ]:
subset.Variable = subset.Variable.str.title().str.replace("_"," ")

In [ ]:
# One df for the actual speeds
subset_speedmph = subset[subset.Variable == 'Speed Mph'].reset_index(drop = True)

In [ ]:
# One df for the percentiles
subset_other= subset[subset.Variable != 'Speed Mph'].reset_index(drop = True)

In [ ]:
def alt_dropdown(df, col_for_dropdown:str, dropdown_menu_title:str):
    # Create dropdown menu
    # Exclude "none" operators which are only scheduled data
    df = df.loc[df[col_for_dropdown] != "None"][[col_for_dropdown]]
    dropdown_list = df[col_for_dropdown].unique().tolist()
    
    # Show only first operator by default
    initialize_first_op = sorted(dropdown_list)[0]
    input_dropdown = alt.binding_select(options=sorted(dropdown_list), name=dropdown_menu_title)
    
    selection = alt.selection_single(name= dropdown_menu_title,fields=[col_for_dropdown],
    bind=input_dropdown, init={col_for_dropdown: initialize_first_op})
                 
    return selection

In [ ]:
selection_test = alt_dropdown(subset, "Route", "Operator/Shape Array")

In [ ]:
def create_jitter_plot(df):
    
    #title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    #inline = df['Loop Or Inlining'].iloc[0]
    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q",
                    scale=alt.Scale(domain=[0,50]),
                    title = "Speed (MPH)",
                    axis=alt.Axis(labelAngle=360, grid=False,)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Sorted Stop Seq:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="top",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="Speeds by Operator-Shape Array")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 75, 200)
    
    return chart1

In [ ]:
chart1 = create_jitter_plot(subset_speedmph).add_selection(selection_test).transform_filter(selection_test)

In [ ]:
chart2 = (
        alt.Chart(subset_other, width=0.5)
        .mark_circle(size=200)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", 
                    title = "Speed (MPH)",
                    scale=alt.Scale(domain=[0, 50]),
                   axis = alt.Axis(grid = False)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=subset_other.columns.tolist(),
            column=alt.Column(
                "Sorted Stop Seq:N",
                header=alt.Header(
                    labelAngle=360,
                    title = None,
                    titleOrient="top",
                    labelOrient="top",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )
    

In [ ]:
chart2 = threshold_utils.chart_size(chart2, 75, 200)

In [ ]:
chart2 = chart2.add_selection(selection_test).transform_filter(selection_test)

In [ ]:
subset['Route Type'] = "Loop or Inlining: " + subset["Loop Or Inlining"].astype(str)

In [ ]:
# https://github.com/altair-viz/altair/issues/1168
title = alt.Chart(subset).mark_text(dy=-40, size=15, fontWeight='normal').encode(
    text='Route Type:N',
)

In [ ]:
title = threshold_utils.chart_size(title, 20, 20)

In [ ]:
title = title.add_selection(selection_test).transform_filter(selection_test)

In [ ]:
title & (chart1.interactive() & chart2.interactive())

### Draft

In [ ]:
def stop_categories1(row):
        if ((row.total_stops > 0) and (row.total_stops <= p25)):
            return "25th  <= 17 stops"
        elif ((row.total_stops > p25) and (row.total_stops <= p75)):
            return "50th <= 30 stops"
        elif ((row.total_stops > p75) and (row.total_stops <= p95)):
               return "75th <= 50 stops"
        elif ((row.total_stops > p95) and (row.total_stops <= p99)):
               return "95th <= 85 stops"
        elif row.total_stops >= p95:
               return "99th >= 203 stops"
        else:
            return "other"

In [ ]:
def create_jitter_plot(df):
    
    title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    inline = df['Loop Or Inlining'].iloc[0]
    
    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", axis=alt.Axis(labelAngle=360)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"{title_op} - Route Type {inline}")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 40, 250)
    
    return chart1

In [ ]:
chart2 = (
        alt.Chart(anaheim_test, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart2 = threshold_utils.chart_size(chart2,80,300)

In [ ]:
chart2

In [ ]:
chart1 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "stop_sequence:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart1 = threshold_utils.chart_size(chart1,80,300)

In [ ]:
chart1

In [ ]:
def create_dot_plot2(df, col_for_dots: str, 
                    x_axis_col:str, y_axis_col:str,
                   tooltip_cols:list, chart_title:str):
  
    chart = (alt.Chart(df).mark_circle(opacity=1, size = 100).transform_window(
    id='rank()',
    groupby=[col_for_dots]).encode(
    alt.X(f'{x_axis_col}:O', sort='descending', 
          axis=alt.Axis(ticks=False, grid=True)),
    alt.Y(f'{y_axis_col}:N'), 
    color=alt.Color(f"{col_for_dots}:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = tooltip_cols)
             .properties(title = chart_title))
    
    return chart

In [ ]:
chart3 = create_dot_plot1(anaheim_test_other, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_other.columns.tolist(),  'Percentile/Average')

In [ ]:
chart3 = threshold_utils.chart_size(chart3,650,300)

In [ ]:
chart4 = create_dot_plot2(anaheim_test_speedmph, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_speedmph.columns.tolist(), 'Speed per Trip')

In [ ]:
chart4 = threshold_utils.chart_size(chart4,650,300)

In [ ]:
chart4

In [ ]:
chart3 + chart4

In [ ]:
chart7 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart7 = threshold_utils.chart_size(chart7,80,300)

In [ ]:
chart8 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart9 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart8

In [ ]:
chart9 | chart8

In [ ]:
# pip install altair==5.0.0rc3
chart5 = alt.Chart(anaheim_test_speedmph, title='Normally distributed jitter').mark_circle(size=50).encode(
    y="rounded_speed:Q",
    x="stop_sequence:N",
    yOffset="jitter:Q",
    color=alt.Color('stop_sequence:Q').legend(None)
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)


In [ ]:
chart5 = threshold_utils.chart_size(chart5,650,300)

In [ ]:
chart5